<a href="https://it-omscholing.nl/locaties/hogeschool-rotterdam/">
<div>
<a><img src='pics/banner.PNG'/></a>
</div>
<div>
<a href=''><img src='pics/miw.PNG'/></a>
<em>Author: Jeroen Boogaard</em>
</div>
</a>

<h1>Practicum Basis Statistiek / SQL </h1>

<h2>Imports<h2>

In [1]:
import os
os.environ['export PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] ='notebook'
os.environ['PYARROW_IGNORE_TIMEZONE'] = '1'

In [2]:
from matplotlib import pyplot as plt
from matplotlib_venn import venn2, venn3
from PIL import Image
from pyspark import SparkContext as sc
from pyspark.sql import SparkSession
import pandas as pd
import pyspark.pandas as ps
import sqlite3

<h2>Extract, Transform and Load</h2>

<h3>Extract</h3>

In [31]:
# https://raw.githubusercontent.com/ajturner/acetate/master/places/Countries-Europe.csv
eurCsvFile = "../csv/countries_europe.csv"
euCsvFile = "../csv/european_union.csv"

 # Create Pandas DataFrames
eurDF = pd.read_csv(eurCsvFile)
euDF = pd.read_csv(euCsvFile)
print(eurDF)
# print(euDF)

    zoom                    name  abbreviation Code  ISO numeric  \
0      3                 Ukraine          Ukr.  UKR          804   
1      3                  France           Fr.  FRA          250   
2      3                   Spain         Spain  ESP          724   
3      3                  Sweden          Swe.  SWE          752   
4      3                 Germany          Ger.  DEU          276   
5      3                 Finland          Fin.  FIN          246   
6      3                  Norway          Nor.  NOR          578   
7      3                  Poland          Pol.  POL          616   
8      3                   Italy         Italy  ITA          380   
9      3          United Kingdom          U.K.  GBR          826   
10     3                 Romania          Rom.  ROU          642   
11     3                 Belarus         Bela.  BLR          112   
12     3                  Greece          Grc.  GRC          300   
13     3                Bulgaria          Blg.  

<h3>Transform</h3>

<h3>Exercise 1</h3>
<h4>Gegeven</h4>

In [4]:
benelux = {"Belgium", "The Netherlands", "Luxembourg"}
nordics = {"Denmark", "Finland", "Iceland", "Norway"}
baltics = {"Estonia", "Latvia", "Lithuania"}

<h4>Gevraagd</h4>
<ol>
    <li>Voeg de kolom <u>group</u> toe aan de dataset <u>eurDF</u> met daarin de groep waartoe een land behoort volgens bovenstaande sets</li>
    <li>Ga na of er nog meer landengroepen zijn binnen Europa en voeg die ook toe</li>
    <li>Sla deze verrijkte dataset op in een nieuw CSV bestand</li>
</ol>

<h4>Oplossing</h4>

In [29]:
eurDF.set_index('name')
print(eurDF)


         zoom                    name  abbreviation     Code ISO numeric  \
0           3                 Ukraine          Ukr.      UKR         804   
1           3                  France           Fr.      FRA         250   
2           3                   Spain         Spain      ESP         724   
3           3                  Sweden          Swe.      SWE         752   
4           3                 Germany          Ger.      DEU         276   
5           3                 Finland          Fin.      FIN         246   
6           3                  Norway          Nor.      NOR         578   
7           3                  Poland          Pol.      POL         616   
8           3                   Italy         Italy      ITA         380   
9           3          United Kingdom          U.K.      GBR         826   
10          3                 Romania          Rom.      ROU         642   
11          3                 Belarus         Bela.      BLR         112   
12          

In [27]:
eurDF['name'].at['Belgium']=0
print(eurDF['name'])
print()

0                      Ukraine
1                       France
2                        Spain
3                       Sweden
4                      Germany
5                      Finland
6                       Norway
7                       Poland
8                        Italy
9               United Kingdom
10                     Romania
11                     Belarus
12                      Greece
13                    Bulgaria
14                     Iceland
15                    Portugal
16              Czech Republic
17                     Denmark
18                     Hungary
19                      Serbia
20                     Austria
21                     Ireland
22                   Lithuania
23                      Latvia
24                     Croatia
25      Bosnia and Herzegovina
26                    Slovakia
27                     Estonia
28                 Netherlands
29                 Switzerland
30                     Moldova
31                     Belgium
32      

In [28]:
eurDF.loc['name']='Belgium'
x= eurDF[eurDF['name']=='Belgium']
x['group']='benelux'
print(eurDF.index)



Index([     0,      1,      2,      3,      4,      5,      6,      7,      8,
            9,     10,     11,     12,     13,     14,     15,     16,     17,
           18,     19,     20,     21,     22,     23,     24,     25,     26,
           27,     28,     29,     30,     31,     32,     33,     34,     35,
           36,     37,     38,     39,     40,     41,     42,     43,     44,
           45,     46, 'name'],
      dtype='object')


C:\Users\marcr\AppData\Local\Temp\ipykernel_28932\2827067655.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['group']='benelux'


In [ ]:
# Oplossing
i=eurDF.loc['name')

#print(eurDF.keys())
eurDF.name =='Belgium'.index(True)
#regionstr in ['benelux','nordics','baltics']:
#    regmem= eval(a) 
    

#eurDF

ValueError: No axis named Belgium for object type DataFrame

<h2>SQLite</h2>

<h3>Load</h3>

<p>Maak een lege database</p>

In [ ]:
!sqlite3 ../db/europe.db "VACUUM;"

In [ ]:
# Connect to SQLite database
dbConnection = sqlite3.connect('../db/europe.db')

<p>Maak tabellen o.b.v. de dataset</p>

In [ ]:
eurDF.to_sql('countries', dbConnection, if_exists='replace', index = False)
euDF.to_sql('eu', dbConnection, if_exists='replace', index = False)

In [ ]:
dbCursor = dbConnection.cursor()

query = """
    SELECT *
    FROM countries
    INNER JOIN eu
    ON countries.Code = eu.Code
    LIMIT 10
"""

result = dbCursor.execute(query).fetchall()
print(result)
print(type(result))

In [ ]:
countriesQuery = """
    SELECT *
    FROM countries
    LIMIT 10
"""

In [ ]:
result = dbCursor.execute(countriesQuery).fetchall()
print(result)

<h2>Afhankelijke en onafhankelijke variabelen</h2>

<h3>Overlappende uitkomsten</h3>

In [ ]:
eu = {"Austria", "Belgium", "Bulgaria", "Croatia", "Cyprus", "Czechia", "Denmark", "Estonia", "Finland", "France", "Germany", "Greece", "Hungary", "Ireland", "Italy", "Latvia", "Lithuania", "Luxembourg", "Malta", "The Netherlands", "Poland", "Portugal", "Romania", "Slovakia", "Slovenia", "Spain", "Sweden"}

In [ ]:
venn3([benelux, baltics, eu], ('Benelux', 'Baltics', 'EU'))

<h3>Exercise 2</h3>
<p>
    Gebruik SQL queries voor het verkrijgen van
<ol>
    <li>P(Nordics); de kans dat een land onderdeel is van de Nordics</li>
    <li><u>P(EU)</u>; de kans dat een Europees land lid is van de Europese Unie</li>
    <li><u>P(EU and Nordics)</u>; de kans dat een land onderdeel is van de Norics en lid is van de EU</li>
</ol>
</p>

In [ ]:
# Oplossing

In [ ]:
# Close connection to SQLite database
dbConnection.close()